In [1]:
from py2neo import Graph, Node, Relationship

def write_into_memgraph(data):
    # Connect to Memgraph
    graph = Graph("bolt://localhost:7687")

    # Extract data from the data
    user_id = data["userId"]
    movie_id = data["movie"]["movieId"]
    movie_title = data["movie"]["title"]
    genres = data["movie"]["genres"]
    rating_value = data["rating"]
    timestamp = data["timestamp"]

    # Create User node if it doesn't exist
    user_node = graph.nodes.match("User", user_id=user_id).first()
    if not user_node:
        user_node = Node("User", user_id=user_id)
        graph.create(user_node)

    # Create Movie node if it doesn't exist
    movie_node = graph.nodes.match("Movie", movie_id=movie_id).first()
    if not movie_node:
        movie_node = Node("Movie", movie_id=movie_id, title=movie_title)
        graph.create(movie_node)

    # Create Rating node if it doesn't exist
    rating_node = graph.nodes.match("Rating", value=rating_value).first()
    if not rating_node:
        rating_node = Node("Rating", value=rating_value)
        graph.create(rating_node)

    # Create Category nodes and BELONGS_TO relationships
    for genre in genres:
        category_node = graph.nodes.match("Category", name=genre).first()
        if not category_node:
            category_node = Node("Category", name=genre)
            graph.create(category_node)
        graph.merge(Relationship(movie_node, "BELONGS_TO", category_node))

    # Create relationships between nodes
    graph.create(Relationship(user_node, "RATED", movie_node, timestamp=timestamp))
    graph.create(Relationship(user_node, "GAVE", rating_node, timestamp=timestamp))
    graph.create(Relationship(rating_node, "HAS", movie_node))

# Example usage
example_data = {'movie': {'genres': ['Comedy', 'Romance'], 'movieId': '111', 'title': 'Truth About Cats & Dogs, The (1996)'}, 'rating': '4', 'timestamp': '881251793', 'userId': '196'}


write_into_memgraph(example_data)


In [2]:
from py2neo import Graph

def read_from_memgraph():
    # Connect to Memgraph
    graph = Graph("bolt://localhost:7687")

    # Cypher query to retrieve user, movie, and associated genres
    cypher_query = """
    MATCH (u:User)-[r:RATED]->(m:Movie)-[:BELONGS_TO]->(g:Category)
    RETURN u.userId AS user_id, m.title AS movie_title, COLLECT(g.name) AS genres
    """

    result = graph.run(cypher_query)

    # Process the result
    for record in result:
        user_id = record['user_id']
        movie_title = record['movie_title']
        genres = record['genres']
        print(f"User ID: {user_id} - Movie Title: {movie_title} - Genres: {', '.join(genres)}")

# Call the function to read data
read_from_memgraph()


User ID: None - Movie Title: Truth About Cats & Dogs, The (1996) - Genres: Comedy, Romance
